In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

import torch
print(torch.backends.mps.is_available())


True


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification",
                model="j-hartmann/emotion-english-distilroberta-base",
                top_k=None,
                device="mps"
       )

pipe("I love this!")

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use mps


[[{'label': 'surprise', 'score': 0.48696979880332947},
  {'label': 'neutral', 'score': 0.2234414517879486},
  {'label': 'joy', 'score': 0.14913064241409302},
  {'label': 'anger', 'score': 0.07174026966094971},
  {'label': 'sadness', 'score': 0.04664652422070503},
  {'label': 'disgust', 'score': 0.01629774644970894},
  {'label': 'fear', 'score': 0.005773560609668493}]]

In [3]:
pipe(books["description"][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985200345516205},
  {'label': 'sadness', 'score': 0.11640875786542892},
  {'label': 'surprise', 'score': 0.02070065401494503},
  {'label': 'disgust', 'score': 0.019100766628980637},
  {'label': 'joy', 'score': 0.015161258168518543},
  {'label': 'anger', 'score': 0.003935154993087053}]]

In [4]:
pipe(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296027541160583},
  {'label': 'neutral', 'score': 0.1403856724500656},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.047942448407411575},
  {'label': 'anger', 'score': 0.009156345389783382},
  {'label': 'disgust', 'score': 0.0026284728664904833},
  {'label': 'sadness', 'score': 0.002122161677107215}],
 [{'label': 'neutral', 'score': 0.44937166571617126},
  {'label': 'disgust', 'score': 0.2735905349254608},
  {'label': 'joy', 'score': 0.10908280313014984},
  {'label': 'sadness', 'score': 0.09362749755382538},
  {'label': 'anger', 'score': 0.040478236973285675},
  {'label': 'surprise', 'score': 0.02697022631764412},
  {'label': 'fear', 'score': 0.006879068911075592}],
 [{'label': 'neutral', 'score': 0.6462165713310242},
  {'label': 'sadness', 'score': 0.24273279309272766},
  {'label': 'disgust', 'score': 0.04342265799641609},
  {'label': 'surprise', 'score': 0.028300516307353973},
  {'label': 'joy', 'score': 0.0142114

In [5]:
sentences = books["description"][0].split(".")
predictions = pipe(sentences)

In [6]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [7]:
predictions[0]

[{'label': 'surprise', 'score': 0.7296027541160583},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'sadness', 'score': 0.002122161677107215}]

In [8]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [9]:
predictions[3]

[{'label': 'fear', 'score': 0.9281684160232544},
 {'label': 'anger', 'score': 0.0321907177567482},
 {'label': 'neutral', 'score': 0.01280868612229824},
 {'label': 'sadness', 'score': 0.008756847120821476},
 {'label': 'surprise', 'score': 0.008597885258495808},
 {'label': 'disgust', 'score': 0.00843180064111948},
 {'label': 'joy', 'score': 0.0010455804876983166}]

In [10]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156345389783382},
 {'label': 'disgust', 'score': 0.0026284728664904833},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.047942448407411575},
 {'label': 'neutral', 'score': 0.1403856724500656},
 {'label': 'sadness', 'score': 0.002122161677107215},
 {'label': 'surprise', 'score': 0.7296027541160583}]

In [11]:
import numpy as np

In [12]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [13]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [14]:
emotion_scores

{'anger': [np.float64(0.051973048597574234),
  np.float64(0.6126183867454529),
  np.float64(0.051973048597574234),
  np.float64(0.3514830768108368),
  np.float64(0.08141231536865234),
  np.float64(0.051973048597574234),
  np.float64(0.23222483694553375),
  np.float64(0.5381841063499451),
  np.float64(0.051973048597574234),
  np.float64(0.12688316404819489)],
 'disgust': [np.float64(0.2735905349254608),
  np.float64(0.348285436630249),
  np.float64(0.15766695141792297),
  np.float64(0.15766695141792297),
  np.float64(0.18449489772319794),
  np.float64(0.15766695141792297),
  np.float64(0.7271748185157776),
  np.float64(0.15766695141792297),
  np.float64(0.15766695141792297),
  np.float64(0.15766695141792297)],
 'fear': [np.float64(0.9281684160232544),
  np.float64(0.942527711391449),
  np.float64(0.9723208546638489),
  np.float64(0.3607070744037628),
  np.float64(0.09504328668117523),
  np.float64(0.038786713033914566),
  np.float64(0.038786713033914566),
  np.float64(0.747427761554718)

In [15]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = pipe(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5252/5252 [05:46<00:00, 15.14it/s]


In [16]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [18]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.051973,0.273591,0.928168,0.932798,0.646217,0.967158,0.729603,9780002005883
1,0.612618,0.348285,0.942528,0.704422,0.887940,0.074825,0.252545,9780002261982
2,0.051973,0.157667,0.972321,0.767237,0.608933,0.074825,0.046931,9780006178736
3,0.351483,0.157667,0.360707,0.251881,0.732687,0.074825,0.046931,9780006280897
4,0.081412,0.184495,0.095043,0.035207,0.925904,0.475881,0.046931,9780006280934
...,...,...,...,...,...,...,...,...
5247,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656,9788172235222
5248,0.057145,0.157667,0.038787,0.400263,0.883199,0.074825,0.227765,9788173031014
5249,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625,9788179921623
5250,0.051973,0.157667,0.459269,0.759455,0.951104,0.368110,0.067808,9788185300535


In [19]:
books = pd.merge(books, emotions_df, on="isbn13")

In [20]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.051973,0.273591,0.928168,0.932798,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.612618,0.348285,0.942528,0.704422,0.887940,0.074825,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.051973,0.157667,0.972321,0.767237,0.608933,0.074825,0.046931
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.351483,0.157667,0.360707,0.251881,0.732687,0.074825,0.046931
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.081412,0.184495,0.095043,0.035207,0.925904,0.475881,0.046931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5247,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.148208,0.030643,0.919165,0.255170,0.853722,0.980877,0.030656
5248,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.057145,0.157667,0.038787,0.400263,0.883199,0.074825,0.227765
5249,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625
5250,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.051973,0.157667,0.459269,0.759455,0.951104,0.368110,0.067808


In [22]:
books.to_csv("books_with_emotions.csv", index=False)